# env lib imports

In [32]:
# pip install pdfminer.six
# pip install -U langchain-community
# !pip install pypdf
# pip install chromadb

# RAG


## import dependencies

In [ ]:
import os
import torch
import transformers
# from pathlib import Path
# from xml.dom.minidom import Document
# import langchain_community, langchain
# from langchain.vectorstores import Chroma
# from langchain.prompts import ChatPromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

import uuid
import chromadb
from json import load
from llama_index.readers.file import PyMuPDFReader
from sentence_transformers import SentenceTransformer

# provide path to a JSON with the API key
from huggingface_hub import login
HUGGING_FACE_API_KEY = load(open('../../config.json'))['HUGGING_FACE_API_KEY']
login(token=HUGGING_FACE_API_KEY)

import json
with open('../../config.json') as config_file: config = json.load(config_file)
KEY_TOGETHERAI = config['KEY_TOGETHERAI']

## Paths
Have a folder named file_dump to dump all the pdf of the institute 

In [10]:
CHROMA_PATH = "institute"  # Name of your vector database
DUMP_FOLD_PATH = "data/file_dump"  # Name of your dump folder

In [11]:
if not os.path.exists(DUMP_FOLD_PATH):
    os.makedirs(DUMP_FOLD_PATH)

## Helper Functions

In [9]:
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

# Function to download a PDF
def download_pdf(pdf_url, save_path):
    response = requests.get(pdf_url)
    if response.status_code == 200:
        with open(save_path, 'wb') as file:
            file.write(response.content)
            print(f"Downloaded: {save_path}")
    else:
        print(f"Failed to download: {pdf_url}")

# Main scraping function
def scrape_pdfs_from_website(base_url, output_folder):
    # Make HTTP request to the base URL
    response = requests.get(base_url)
    if response.status_code != 200:
        print(f"Failed to access {base_url}")
        return
    
    # Parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all anchor tags with PDF links
    pdf_links = []
    for link in soup.find_all('a', href=True):
        href = link['href']
        if href.endswith('.pdf'):
            pdf_links.append(urljoin(base_url, href))  # Convert relative links to absolute
    
    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)
    
    # Download all PDF files
    for pdf_url in pdf_links:
        pdf_name = os.path.basename(pdf_url)
        save_path = os.path.join(output_folder, pdf_name)
        download_pdf(pdf_url, save_path)

In [12]:
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

# Function to scrape text from a page and save it
def scrape_page_text(page_url, output_folder, file_name):
    try:
        response = requests.get(page_url)
        if response.status_code != 200:
            print(f"Failed to access {page_url}")
            return

        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Extract all text
        page_text = soup.get_text(separator='\n').strip()
        
        # Save text to a file
        os.makedirs(output_folder, exist_ok=True)
        file_path = os.path.join(output_folder, f"{file_name}.txt")
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(page_text)
            print(f"Saved: {file_path}")
    except Exception as e:
        print(f"Error scraping {page_url}: {e}")

# Function to find all links on the main page
def scrape_website(base_url, output_folder):
    try:
        response = requests.get(base_url)
        if response.status_code != 200:
            print(f"Failed to access {base_url}")
            return

        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Find all page links (anchor tags with href)
        page_links = []
        for link in soup.find_all('a', href=True):
            href = link['href']
            # Filter to avoid external links or specific conditions (if needed)
            if href.startswith('/'):  # Adjust as per website structure
                full_url = urljoin(base_url, href)
                page_links.append(full_url)
        
        # Scrape each page and save text
        for i, page_url in enumerate(page_links):
            file_name = f"page_{i+1}"  # Custom file naming
            scrape_page_text(page_url, output_folder, file_name)
    except Exception as e:
        print(f"Error processing {base_url}: {e}")

# # Example Usage
# base_url = "https://example.com"  # Replace with your target website
# output_folder = "website_text"
# scrape_website(base_url, output_folder)

# Example Usage
base_url = "https://www.iiitd.ac.in/academics/resources"
output_folder = DUMP_FOLD_PATH
scrape_pdfs_from_website(base_url, output_folder)

Downloaded: data/file_dump/Final Placement Brochure 2024-25.pdf
Downloaded: data/file_dump/BTech-Ordinances.pdf
Downloaded: data/file_dump/2024-May-UG Regulations.pdf
Downloaded: data/file_dump/2021-Jan-UG%20Regulations.pdf
Downloaded: data/file_dump/2019-August-UG-Regulations.pdf
Downloaded: data/file_dump/2019-July-UG-Regulations.pdf
Downloaded: data/file_dump/2017-July-UG-Regulations.pdf
Downloaded: data/file_dump/2024-May-BTech(CSE)-Regulations.pdf
Downloaded: data/file_dump/2021-Jan-BTech(CSE)-Regulations.pdf
Downloaded: data/file_dump/2019%20-August-BTech(CSE)-Regulations.pdf
Downloaded: data/file_dump/2019-July-BTech(CSE)-Regulations.pdf
Downloaded: data/file_dump/2017%20-July-BTech(CSE)-Regulations.pdf
Downloaded: data/file_dump/2016%20-BTech(CSE)-Regulations.pdf
Downloaded: data/file_dump/2024-May-BTech(ECE)-Regulations.pdf
Downloaded: data/file_dump/2021-Jan-BTech(ECE)-Regulations.pdf
Downloaded: data/file_dump/2019%20-August-BTech(ECE)-Regulations.pdf
Downloaded: data/file_d

In [69]:
def __load_documents(DUMP_FOLD_PATH):
    pdf_paths = [f for f in os.listdir(DUMP_FOLD_PATH) if f.endswith('.pdf')]
    txt_paths = [f for f in os.listdir(DUMP_FOLD_PATH) if f.endswith('.txt')]
    documents = []  # List of documents
    for pdf_path in pdf_paths: 
        chunks, doc_idxs = __extract_text_from_pdf(pdf_path, loader="PyMuPDFReader")
        documents.append(chunks)
    for txt_path in txt_paths:
        with open(os.path.join(DUMP_FOLD_PATH, txt_path), 'r') as f:
            documents.append(f.read())
    return documents
def __process_documents(documents):
    # documents is list of list of chunks i want a only list of chunks
    documents = [chunk for doc in documents for chunk in doc]

    # Chunck processing
    documents = [chunk for chunk in documents if chunk != ""]
    documents = [chunk.replace("\n", "") for chunk in documents]
    return documents

def __extract_text_from_pdf(file_path, folder_path=DUMP_FOLD_PATH, loader=None):
    file_path = os.path.join(folder_path, file_path)

    if loader == "PyPDF2":
        loader = PyPDFLoader(file_path)
        pages = loader.load()

        # Split document into chunks (e.g., chunk_size=500)
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
        chunks = text_splitter.split_documents(pages)
        return chunks, None

    elif loader == "PyMuPDFReader":
        loader = PyMuPDFReader()
        documents = loader.load(file_path=file_path)

        # Text splitter to split the document
        from llama_index.core.node_parser import SentenceSplitter
        text_parser = SentenceSplitter(
            chunk_size=1024,
            # separator=" ",
        )

        text_chunks = []
        # maintain relationship with source doc index, to help inject doc metadata in (3)
        doc_idxs = []
        for doc_idx, doc in enumerate(documents):
            cur_text_chunks = text_parser.split_text(doc.text)
            text_chunks.extend(cur_text_chunks)
            doc_idxs.extend([doc_idx] * len(cur_text_chunks))
        
        return (text_chunks, doc_idxs)
    else:
        raise ValueError("Invalid loader")

def __create_collection(collection_name=CHROMA_PATH):
    chroma_client = chromadb.Client()
    # collection = chroma_client.create_collection(name=CHROMA_PATH)
    collection = chroma_client.get_or_create_collection(name=collection_name)
    embedder = SentenceTransformer('all-MiniLM-L6-v2')
    return collection, embedder
def __add_documents_to_collection(documents, rag_objects):
    """
    Args:
        documents: list of documents (str)
        rag_objects: tuple of (collection, embedder)
        
    Returns:
        collection: collection of documents
    """
    collection, embedder = rag_objects

    # Dynamically retrieve the maximum batch size from the server
    max_batch_size = collection._client.get_max_batch_size()

    embeddings = embedder.encode(documents)
    ids = [str(uuid.uuid4()) for _ in documents]  # Adjust this for actual document IDs/names

    # Split data into batches
    for i in range(0, len(documents), max_batch_size):
        batch_docs = documents[i:i + max_batch_size]
        batch_embeddings = embeddings[i:i + max_batch_size]
        batch_ids = ids[i:i + max_batch_size]

        # Perform the upsert operation for each batch
        collection.upsert(
            ids=batch_ids,
            embeddings=batch_embeddings.tolist(),
            documents=batch_docs
        )

    return collection, embedder


def __pipline_gen(model_id = "meta-llama/Llama-3.2-3B-Instruct", 
                bits_rate=4, device_map={"": "cuda:0"}, torch_dtype=torch.float32,
                pipeline_task="text-generation"):
    quant_config = BitsAndBytesConfig(bit_rate=bits_rate)
    llm_model_llama = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=quant_config,
        device_map=device_map,  # Adjust as needed
        torch_dtype=torch_dtype,  # Use mixed precision
    )

    tokenizer_llama = AutoTokenizer.from_pretrained(model_id)

    pipeline_llama = transformers.pipeline(
        task=pipeline_task,
        model=llm_model_llama,
        tokenizer=tokenizer_llama,
        output_hidden_states=True,  # Ensure the model outputs hidden states
        # device=0                  # Ensure this matches your GPU device ID
    )

    tokenizer_llama.pad_token_id = tokenizer_llama.eos_token_id
    llm_model_llama.pad_token_id = tokenizer_llama.eos_token_id

    llm_model_llama = pipeline_llama.model; tokenizer_llama = pipeline_llama.tokenizer 
    llm_model_llama.eval()
    return pipeline_llama

def __retrieve_relevant_documents(query, collection, embedder, k=5):
    query_embedding = embedder.encode([query])
    results = collection.query(
        query_embeddings=query_embedding.tolist(),
        n_results=k
    )
    documents = [doc[0] if isinstance(doc, list) else doc for doc in results['documents']]
    
    return documents

### Together AI call for RAG

In [57]:
from together import Together
client = Together(api_key=KEY_TOGETHERAI)

In [58]:
def qa_together(query = "ohh hii", model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo",
                collection = None, embedder = None,
                max_token = 512,
                max_new_tokens = 1024,
                temperature = 0.7,
                top_p = 0.7,
                top_k = 50,
                stop = ["<|eot_id|>","<|eom_id|>"],
                repitition_penalty = 1,
                stream = True,
                answer = None,
                file_name = "output.txt",
                verbose = 0):        
    try:
        relevant_docs = __retrieve_relevant_documents(query, collection=collection, embedder=embedder, k=5)
        context = "\n".join(relevant_docs)
        prompt = f"Context: {context}\n\nQ: {query}\nA:"
        response = client.chat.completions.create(
                model=model_name,
                messages=[
                    {
                    "role": "user",
                    "content": f"{prompt}"
                    }
                ],
                max_new_tokens=max_new_tokens,
        )
        if verbose > 1: 
            with open(file_name, "a") as f: 
                f.write(f"PROMT{{{prompt}}}\n")
                f.write(f"CORRECT_ANSWER{{{answer}}}\n")
                f.write(response.choices[0].message.content)
        if verbose > 0: print(response.choices[0].message.content)
        return response.choices[0].message.content
    except Exception as e: 
        print(f"Error in response {e}")

## RAG Collection creation and populating

In [70]:
collection, embedder = __add_documents_to_collection(
                            documents =  __process_documents(__load_documents(DUMP_FOLD_PATH)), 
                            rag_objects=__create_collection(),
                            # max_batch_size=10**12
                        )

In [77]:
# # pickle dump collection, embedder    
# import pickle
# with open('collection.pkl', 'wb') as f:
#     pickle.dump(collection, f)
# with open('embedder.pkl', 'wb') as f:
#     pickle.dump(embedder, f)

In [ ]:
# # load collection, embedder
# import pickle
# with open('collection.pkl', 'rb') as f:
#     collection = pickle.load(f)
# with open('embedder.pkl', 'rb') as f:
#     embedder = pickle.load(f)

In [76]:
query = "IIITD B.Tech. program is broadly divided in two halves, tell about them ?"
query = "What are reqirement for CSD program for graduation in IIITD ?"
query = "What are requirements for Graduation; For a B.Tech. (CSD) degree, a student must satisfy which all the following requirements:"
qa_together(query = query, collection=collection, embedder=embedder, max_new_tokens=32, verbose=1)

I'm assuming the context is related to a B.Tech. (CSD) degree, which is likely related to Computer Science and Design. However, without more information about the specific institution or course, I'll provide a general outline of common requirements for a B.Tech. degree:

For a B.Tech. (CSD) degree, a student must satisfy the following requirements:

1. **Credit Requirements**: Complete a minimum of 160-180 credits, depending on the institution, with a distribution of credits across various subjects, including core, elective, and lab courses.
2. **Core Courses**: Complete all the core courses specified by the institution, which typically include subjects like mathematics, computer science, and engineering fundamentals.
3. **Elective Courses**: Complete a set of elective courses, which may include specializations like data science, artificial intelligence, machine learning, or software engineering.
4. **Lab Courses**: Complete a set of lab courses, which provide hands-on experience in pr

"I'm assuming the context is related to a B.Tech. (CSD) degree, which is likely related to Computer Science and Design. However, without more information about the specific institution or course, I'll provide a general outline of common requirements for a B.Tech. degree:\n\nFor a B.Tech. (CSD) degree, a student must satisfy the following requirements:\n\n1. **Credit Requirements**: Complete a minimum of 160-180 credits, depending on the institution, with a distribution of credits across various subjects, including core, elective, and lab courses.\n2. **Core Courses**: Complete all the core courses specified by the institution, which typically include subjects like mathematics, computer science, and engineering fundamentals.\n3. **Elective Courses**: Complete a set of elective courses, which may include specializations like data science, artificial intelligence, machine learning, or software engineering.\n4. **Lab Courses**: Complete a set of lab courses, which provide hands-on experien

In [42]:
# pipeline = __pipline_gen()

# def generate_rag_answer(query, pipe=pipeline, 
#                         max_length=80, max_new_tokens=75, temperature=0.01, top_p=1.0, top_k=50,
#                         verbose=0, 
#                         ):
#     relevant_docs = __retrieve_relevant_documents(query, collection=collection, embedder=embedder, k=5)
#     context = "\n".join(relevant_docs) 
#     prompt = f"Context: {context}\n\nQ: {query}\nA:"
#     result = pipe(
#         prompt,
#         # max_length=max_length,
#         max_new_tokens=max_new_tokens,
#         do_sample=True,
#         temperature=temperature,
#         top_p=top_p,
#         top_k=top_k
#     )
#     answer = result[0]['generated_text'].replace(prompt, '').strip()
#     if verbose: print(f"Raw answer: {result}")
#     return answer

# generate_rag_answer("Tell me about academic calender?")